In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install pycham

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00


# 참조 카운터

- 메모리 관리의 기본

In [8]:
class obj:
  pass
def f(x):
  print(x)

a=obj()
f(a)
a=None

<순환 참조>

참조 카운트로는 해결할 수 없는 문제

a = b = c = None 해도 메모리에서 삭제가 안됌

<GC>

- 메모리가 부족해지는 시점에서 자동호출

- 메모리 해제를 GC에 미루다 보면 메모리 사용량이 커짐

- DeZero 개발할 때는 순환참조를 만드지 않는 것이 좋음

In [7]:
a=obj()
b=obj()
c=obj()

a.b = b
a.c = c
a = b = c =None

# Weakref 모듈

In [11]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} in not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0  # generation 을 0으로 초기화

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # set_creator 호출될때 부모 함수의 세대보다 1만큼 큰 값을 설정

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x:x.generation)

      while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None: # 미분값(grad)을 처음 설정하는 경우 출력에서 전해지는 미분값으로 대입
            x.grad = gx      # 처음 이후 부터는 전달된 미분값을 더해주도록 수정
          else:
            x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

In [12]:
import weakref

class Function:
  def __call__(self, inputs):
    xs = [ x.data for x in inputs ]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inputs]) #
    for output in outputs:
      output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
      return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()

# 동작확인

<DeZero 순환 참조 문제 해소 방법>

- for 문이 두 번째 반복될 때 x와 y가 덮어 써짐

- 사용자는 이전의 계산 그래프를 더 이상 참조하지 않게됨

- 참조 카운트가 0이 되므로 이 시점에 계산 그래프에 사용된 메모리가 삭제


# 메모리 절약 모드

## DeZero의 역전파 개선

- 필요없는 미분값 삭제

- 현재의 DeZero에서는 y.backward() 를 실행하면 모든 변수가 미분값을 메모리에 유지함

- 역전파로 구하고 싶은 미분값은 말단 변수 (x0, x1)뿐일 때가 대부분

- 중간 변수에 대해서는 미분값을 제거하는 모드가 추가 필요함.

In [ ]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} in not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0  # generation 을 0으로 초기화

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # set_creator 호출될때 부모 함수의 세대보다 1만큼 큰 값을 설정

  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      if not retain_grad:
        for y in f.outputs:
          y().grad = None

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x:x.generation)

      while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None: # 미분값(grad)을 처음 설정하는 경우 출력에서 전해지는 미분값으로 대입
            x.grad = gx      # 처음 이후 부터는 전달된 미분값을 더해주도록 수정
          else:
            x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

# Config 클래스를 활용한 모드 전환
- 순전파만 수행할 경우를 위한 모드 전환

- Config 클래스

In [13]:
import weakref

class Config:
  enable_backprop = True

class Function:
  def __call__(self, inputs):
    xs = [ x.data for x in inputs ]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs]) #
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()

# with 문을 활용한 모드 전환
- with 블록에 들어갈 때의 처리 (전처리)와 with 블록을 빠져나올 때의 처리(후처리)를 자동으로 수행

- with 문의 원리를 이용하여 ‘역전파 비활성 모드’로 전환하려 함

- with 블록 안에서만 ‘역전파 비활성 모드’, with 블록을 벗어나면 일반모드

using_config 함수 구현

no_grad 함수 구현

In [ ]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    setattr(Config, name, old_value)

def no_grad():
  return using_config('enable_backprop', False)

with no_grad():
  x = Variable(np.array(2.0))
  y = square(x)

with using_config('enable_backprop',False):
  x = Variable(np.array(2.0))
  y = square(x)

In [16]:
class Variable:
  def __init__(self, data, name = None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} in not supported'.format(type(data)))

    self.data = data
    self.name = name
    self.grad = None
    self.creator = None
    self.generation = 0  # generation 을 0으로 초기화

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # set_creator 호출될때 부모 함수의 세대보다 1만큼 큰 값을 설정

  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      if not retain_grad:
        for y in f.outputs:
          y().grad = None

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x:x.generation)

      while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None: # 미분값(grad)을 처음 설정하는 경우 출력에서 전해지는 미분값으로 대입
            x.grad = gx      # 처음 이후 부터는 전달된 미분값을 더해주도록 수정
          else:
            x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

# 연산자 오버로드
- Mul 클래스 구현

In [17]:
class Mul(Function):
  def forward(self, x0,x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0,x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0

def mul(x0,x1):
  return Mul()(x0,x1)

# 곱셈 연사자 * 오버로드
- a * B 실행시 호출 순서

1. 인스턴스 a의 특수 메서드인 __mul__이 호출

2. 연산자 * 왼쪽의 a가 인수 self에 전달되고, 오른쪽의 b가 other에 전달됨

In [ ]:
class Variable:
  def __init__(self, data, name = None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} in not supported'.format(type(data)))

    self.data = data
    self.name = name
    self.grad = None
    self.creator = None
    self.generation = 0  # generation 을 0으로 초기화

  def __mul__(self, other):
    return mul(self, other)

  Variable.__mul__ = mul
  Variable.__add__ = add

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # set_creator 호출될때 부모 함수의 세대보다 1만큼 큰 값을 설정

  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      if not retain_grad:
        for y in f.outputs:
          y().grad = None

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x:x.generation)

      while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None: # 미분값(grad)을 처음 설정하는 경우 출력에서 전해지는 미분값으로 대입
            x.grad = gx      # 처음 이후 부터는 전달된 미분값을 더해주도록 수정
          else:
            x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

# 연산자 오버로드 (ndarray)

<ndarray 인스턴스와 수치 데이터와도 함께 사용하도록 개선>

- a * np.array(2.0) 처럼 ndarray 인스턴스와 함께 사용할 수 없음

- 3 + b 처럼 수치 데이터도 함께 사용할 수 없음

- Variable 인스턴스와 ndarray 인스턴스 함께 사용 가능

- int 나 float 등도 함께 사용 가능

In [18]:
def as_variable(obj):
  if isinstance(obj, Variable):
    return obj

import weakref

class Config:
  enable_backprop = True

class Function:
  def __call__(self, inputs):
    inputs = [as_variable(x) for x in inputs]
    xs = [ x.data for x in inputs ]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs]) #
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()

float, int와 함께 사용하기

as_array 함수 사용

In [19]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

def add(x0,x1):
  x1 = as_array(x1)
  return Add()(x0,x1)


# 연산자 오버로드

- 음수 (부호 변환)

- 뺄셈

- 나눗셈

- 거듭제곱

1. 음수의 미분

▪ 역전파 상류(출력 쪽)에서 전해지는 미분에 -1을 곱하여 하류로 흘려보내 줌

▪ Neg 클래스를 구현한 다음 파이썬 함수로 사용할 수 있도록 neg 함수 구현

▪ __neg__ 에 neg를 대입하면 완성


In [20]:
class Neg(Function):
  def forward(self, x):
    return -x

  def backward(self, gy):
    return -gy

def neg(x):
  return Neg()(x)

Variable.__neg__ = neg

2. 뺄셈의 미분

▪ 역전파는 상류에서 전해지는 미분값에 1을 곱한 값이 x0의 미분 결과가 되면, -1을
곱한 값이 x1의 미분 결과가 됨

▪ x0와 x1이 Variable 인스턴스라면 y = x0 – x1 계산을 수행할 수 있음



In [22]:
class Sub(Function):
  def forward(self, x0,x1):
    y = x0 - x1
    return y

  def backward(self, gy):
    return gy, -gy

def sub(x0,x1):
  x1 = as_array(x1)
  return Sub()(x0,x1)

Variable.__sub__ = sub

def rsub(x0,x1):
  x1 = as_array(x1)
  return sub(x1, x0)
Variable.__rsub__= rsub

3. 나눗셈의 미분

- 나눗셈도 뺄셈과 마찬가지로 좌/우항 주 어느 것에 적용할지에 따라 적용되는 함수가 다름

In [23]:
class Div(Function):
  def forward(self, x0,x1):
    y = x0/x1
    return y

  def backward(self, gy):
    x0,x1 = self.inputs[0].data, self.inputs[1].data
    gx0 = gy / x1
    gx1 = gy * (-x0 / x1 ** 2)
    return gx0, gx1

def div(x0,x1):
  x1 = as_array(x1)
  return Div()(x0,x1)

def rdiv(x0,x1):
  x1 = as_array(x1)
  return div(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

4. 거듭제곱의 미분

- 순전파 메서드인 forward(x)는 밑에 해당하는 x만 받게 함

- 특수 메서드인 __pow__ 에 함수 pow를 할당

In [24]:
class Pow(Function):
  def __inti__(self,c):
    self.c = c

  def forward(self, x):
    y = x ** self.c
    return y

  def backward(self, gy):
    x = self.inputs[0].data
    c = self.c

    gx = c * x ** (c-1) * gy
    return gx

def pow(x,c):
  return Pow(c)(x)

Variable.__pow__ = pow